In [59]:
import os
import librosa
import math
import numpy as np

In [41]:
# list for the array
channels = []
sr = 44100

# Read in data
for root, dirs, files in os.walk("./data"):
    cur_chan = []
    
    for file in files:
        audio, sr = librosa.load(os.path.join(root, file))
        cur_chan.append(audio)
            
    if(cur_chan):
        channels.append(cur_chan)


In [52]:
# length of sine sweep in S
M = 10

# start frequency
f_1 = 20

# end frequency
f_2 = 20000

# harmonic
k = 2

# total length of fundamental in seconds
T =  (((2 * math.pi * M) / (k - 1)) - math.pi / 2) * math.log(f_2 / f_1) / (2 * math.pi * f_1)

print(T)

3.3675306985037916


In [68]:
# Calculate harmonic offsets
harm_off = []
num_harm = 30

for k in range(2,num_harm):
    n = (T * sr * math.log(k)) / math.log(f_2 / f_1)
    harm_off.append(n)
    
print(harm_off)

[7450.898974031561, 11809.395470501762, 14901.797948063122, 17300.451659971306, 19260.294444533323, 20917.317908501456, 22352.696922094685, 23618.790941003524, 24751.350634002873, 25775.87549803699, 26711.193418564886, 27571.602499363107, 28368.21688253302, 29109.84713047307, 29803.595896126244, 30455.27269026429, 31069.689915035084, 31650.878751677257, 32202.249608034428, 32726.71337900322, 33226.774472068544, 33704.603137353406, 34162.09239259645, 34600.90331994261, 35022.50147339467, 35428.18641150529, 35819.11585656458, 36196.32561246085]


In [76]:
# calculate harmonic distortion
THD = []

# position of fundamental
fund_pos = M * sr

for channel in channels:
    cur_chan_thd = []
    
    #calculate the harmonic distortion in each sample for the current channel
    for sample in channel:
        harm_volt = 0
        
        for i in range(num_harm - 3):
            cur_harm = int(fund_pos - harm_off[i] + 10)
            next_harm = int(fund_pos - harm_off[i + 1])
            
            v_rms = 0.707 * np.max(sample[next_harm:cur_harm])
            harm_volt += pow(v_rms, 2)
            
        cur_sam_thd = math.sqrt(harm_volt) / (.707 * np.max(sample))
        
        cur_chan_thd.append(cur_sam_thd)
        
    THD.append(cur_chan_thd)
    
print(THD)

[[0.0016792251969153305, 0.0015426416130973175, 0.0022075767746711403], [0.0019017538084009867, 0.0015064031644488015, 0.0016029438362942401]]
